In [1]:
import pandas as pd
test_df = pd.read_csv('./Dataset/test.csv')
test_img_paths = test_df['img_path'].values

In [2]:
import cv2
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms
    
    def __getitem__(self, index):
        img_path = './Dataset'+self.img_paths[index][1:]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    def __len__(self):
        return len(self.img_paths)   

In [3]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std= (0.229,0.224,0.224), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [4]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [5]:
import torch
from tqdm import tqdm
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [6]:
import os
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
os.environ['CUDA_LAUNCH_BLOCKING']="1"
print(device)
print(f'torch version : {torch.__version__}')

cuda:0
torch version : 1.12.1


In [7]:
import torch.nn as nn
from torchvision import models

class BaseModel(nn.Module):
    def __init__(self, num_classes=50):
        super(BaseModel, self).__init__()
        #self.backbone = models.convnext_large(weights=models.ConvNeXt_Large_Weights.IMAGENET1K_V1)
        self.backbone = models.convnext_small()
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.Dropout(0.5)(x)
        x = self.classifier(x)
        return x
        
import torchsummary
model = BaseModel(num_classes=50)
torchsummary.summary(model, (3,224,224),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       LayerNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,800
           Permute-4           [-1, 56, 56, 96]               0
         LayerNorm-5           [-1, 56, 56, 96]             192
            Linear-6          [-1, 56, 56, 384]          37,248
              GELU-7          [-1, 56, 56, 384]               0
            Linear-8           [-1, 56, 56, 96]          36,960
           Permute-9           [-1, 96, 56, 56]               0
  StochasticDepth-10           [-1, 96, 56, 56]               0
          CNBlock-11           [-1, 96, 56, 56]               0
           Conv2d-12           [-1, 96, 56, 56]           4,800
          Permute-13           [-1, 56, 56, 96]               0
        LayerNorm-14           [-1, 56,

In [8]:
checkpoint = torch.load('./checkpoint/best_convnext_small_0.7872.pth')
# print(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'], strict=False)

<All keys matched successfully>

In [9]:
preds = inference(model, test_loader, device)

100%|██████████| 396/396 [01:21<00:00,  4.87it/s]

Done.


In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_df = pd.read_csv('./Dataset/train.csv')
train_df['artist'] = le.fit_transform(train_df['artist'].values)
print(train_df.head())


   id          img_path  artist
0   0  ./train/0000.jpg       9
1   1  ./train/0001.jpg      48
2   2  ./train/0002.jpg       7
3   3  ./train/0003.jpg      10
4   4  ./train/0004.jpg      24


In [11]:
preds = le.inverse_transform(preds)
print(preds[:5])
submit = pd.read_csv('./Dataset/sample_submission.csv')


['Edgar Degas' 'Amedeo Modigliani' 'Caravaggio' 'Albrecht Du rer'
 'Pablo Picasso']


In [12]:
submit['artist']=preds
submit.head()
submit.to_csv('./Dataset/submit.csv', index=False)